# 总体回测前

## 设置参数

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import datetime
import random

from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

CONF = Config().data[0]
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
CRYPTOCURRENCY = list(CRYPTOCURRENCY.keys())
NASDAQ100 = CONF['NASDAQ100']
HS300 = list(ts.get_hs300s()['code'])

BENCHMARK = '399300'
# TARGET = ['399300']
TARGET = HS300

ALL_TARGET = TARGET[:]

pd

<module 'pandas' from 'd:\\python36\\lib\\site-packages\\pandas\\__init__.py'>

In [2]:
%%time

### 时间设置
# start_date = '2005-01-01'
# start_date = '2006-01-01'
# start_date = '2007-01-01'
# start_date = '2008-01-01'
# start_date = '2009-01-01'
# start_date = '2010-01-01'
# start_date = '2011-01-01'
# start_date = '2012-01-01'
# start_date = '2013-01-01'
# start_date = '2014-01-01'
# start_date = '2015-01-01'
# start_date = '2015-07-01'
# start_date = '2016-01-01'
start_date = '2017-01-01'
# start_date = '2018-01-01'
# start_date = '2018-08-01'
# end_date = '2016-01-01'
# end_date = '2018-08-01'
end_date = time.strftime('%Y-%m-%d')

TURTLE_POS = 10
### Turtle System One - Short
TURTLE_SHORT_BUY_N = 20
TURTLE_SHORT_SELL_N = 20
### Turtle System Two - Long
TURTLE_LONG_BUY_N = 60
TURTLE_LONG_SELL_N = 60

### 业务设置
IS_HAPPYMONEY = False
IS_TAX = False
IS_SLIPPAGE = False
IS_RANDOM_BUY = False
IS_FILTER = False
IS_MARKETUP = True
IS_BUYBENCHMARK = True
IS_SHOWBUYLIST = True
START_MONEY = 10000
HAPPY_MONEY = 0
PROPERTY = START_MONEY
CASH = START_MONEY

Wall time: 0 ns


# 每天开盘前

## 是否交易日

In [3]:
today = datetime.date.today().strftime('%Y-%m-%d')
lastweek = (datetime.date.today()  - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
df = ts.get_k_data(code='399300', index=True, start=lastweek, end=today)
df

if df.iloc[-1].date == today:
    is_today_trading = True
else:
    is_today_trading = False
#     return False

,date,open,close,high,low,volume,code
154,2018-08-20,3238.22,3267.25,3267.25,3209.01,70904228.0,sz399300
155,2018-08-21,3271.84,3326.65,3331.71,3270.03,75205583.0,sz399300
156,2018-08-22,3328.97,3307.95,3328.97,3299.39,50787057.0,sz399300
157,2018-08-23,3308.46,3320.03,3336.11,3285.81,61112334.0,sz399300
158,2018-08-24,3308.48,3325.33,3353.04,3291.87,63765897.0,sz399300
159,2018-08-27,3339.39,3406.57,3406.57,3339.26,94270082.0,sz399300


## 准备历史数据

In [4]:
%%time

stock_df_dict = {}

for symbol in TARGET + [BENCHMARK]:
    stock_data_file = '../database/market/%s_lite.csv' % symbol
    stock_df = pd.read_csv(stock_data_file)

    # 筛选字段
    stock_df = stock_df.loc[:, ['date', 'open', 'close']]

    # 去掉Nasdaq行情首行的当天行情
    if symbol in NASDAQ100:
        stock_df = stock_df.drop([0])

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期
    # 445 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    stock_df = stock_df.assign(date=pd.to_datetime(stock_df['date']))  # need .index.to_period('D')

    # 用日期作索引，日期升序排序
    # 95.1 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    if symbol in NASDAQ100 or symbol in CRYPTOCURRENCY:
        stock_df = stock_df[::-1]
    stock_df.set_index(['date'], inplace=True)
    stock_df.index = stock_df.index.to_period('D')

    # 计算每天涨跌幅
    stock_df['o_pct_chg'] = stock_df.open.pct_change(1)
    stock_df['c_o_pct_chg'] = (stock_df.open - stock_df.close.shift(1)) / stock_df.close.shift(1)
    
    # Turtle指标
    stock_df['ROLLING_%d_MAX' % TURTLE_LONG_BUY_N] = stock_df['open'].rolling(TURTLE_LONG_BUY_N).max()
    stock_df['ROLLING_%d_MIN' % TURTLE_LONG_SELL_N] = stock_df['open'].rolling(TURTLE_LONG_SELL_N).min()
    stock_df['MA180'] = stock_df['open'].rolling(180).mean()
    stock_df['MA60'] = stock_df['open'].rolling(60).mean()
    stock_df['MA30'] = stock_df['open'].rolling(30).mean()
    
    # 减少数据
    # stock_df = stock_df['2016-01-01':]
    stock_df.dropna(how='any', inplace=True)
    
    stock_df_dict[symbol] = stock_df

Wall time: 5.15 s


## 抽查数据

In [5]:
stock_df_dict[BENCHMARK].head(2)
stock_df_dict[BENCHMARK].tail(2)

df = stock_df_dict[BENCHMARK].iloc[:].copy()
df.columns
df.reset_index(drop=False, inplace=True)
df['date'] = df['date'].apply(lambda x: x.to_timestamp().to_datetime64())
df.set_index('date', inplace=True)
display_charts(df, chart_type='stock', kind='line', y=['open', 'ROLLING_60_MAX', 'ROLLING_60_MIN', 'MA60', 'MA180'], figsize=(900, 600))

,open,close,o_pct_chg,c_o_pct_chg,ROLLING_60_MAX,ROLLING_60_MIN,MA180,MA60,MA30
date,,,,,,,,,
2017-09-25,3829.22,3817.79,0.000721,-0.002217,3851.69,3611.9,3548.587111,3748.519167,3801.640333
2017-09-26,3810.33,3820.78,-0.004933,-0.001954,3851.69,3611.9,3551.344778,3751.244167,3805.458667


,open,close,o_pct_chg,c_o_pct_chg,ROLLING_60_MAX,ROLLING_60_MIN,MA180,MA60,MA30
date,,,,,,,,,
2018-08-23,3308.46,3320.03,-0.006161,0.000154,3846.94,3238.22,3841.866222,3510.0745,3411.291667
2018-08-24,3308.48,3325.33,0.000006,-0.003479,3846.94,3238.22,3837.835500,3502.7275,3405.291000


Index(['open', 'close', 'o_pct_chg', 'c_o_pct_chg', 'ROLLING_60_MAX',
       'ROLLING_60_MIN', 'MA180', 'MA60', 'MA30'],
      dtype='object')

## 准备订单数据

In [6]:
# get_order

## 关注临界卖出目标

## 关注临界买入目标

In [7]:
watch_buy_df = None
watch_buy_df = pd.DataFrame(columns=[
    'date', 'symbol', 'open', 'ROLLING_MAX', 'minus'
])

today = pd.to_datetime(time.strftime('%Y-%m-%d'))
today = today.to_period(freq='D')
'today', today

for symbol in TARGET + [BENCHMARK]:
    if len(stock_df_dict[symbol]) == 0:
        continue
    yesterday_market = stock_df_dict[symbol].iloc[-1]
    if today - yesterday_market.name > 3:
        continue
    watch_buy_df = watch_buy_df.append(
        {
            'date': yesterday_market.name,
            'symbol': symbol,
            'open': yesterday_market.open,
            'ROLLING_MAX': yesterday_market['ROLLING_%d_MAX' % TURTLE_LONG_BUY_N],
            'minus': (yesterday_market['ROLLING_%d_MAX' % TURTLE_LONG_BUY_N] - yesterday_market.open) / yesterday_market.open
        },
        ignore_index=True
    )

# watch_buy_df.sort_values(by='date')
watch_buy_df.sort_values(by='minus')[:10]

('today', Period('2018-08-27', 'D'))

,date,symbol,open,ROLLING_MAX,minus
72,2018-08-24,600583,5.89,5.908,0.003056
17,2018-08-24,600050,5.66,5.680,0.003534
106,2018-08-24,601006,8.97,9.040,0.007804
44,2018-08-24,600339,4.74,4.780,0.008439
41,2018-08-24,600297,6.74,6.830,0.013353
188,2018-08-24,000157,3.95,4.007,0.014430
240,2018-08-24,002153,35.87,36.390,0.014497
155,2018-08-24,601857,8.06,8.180,0.014888
161,2018-08-24,601898,4.95,5.040,0.018182
126,2018-08-24,601229,11.23,11.450,0.019590


# 每天交易时

## 第一时间获取开盘价

In [8]:
%%time

today_open_market = ts.get_today_all()
print('')
print('len(today_open_market)', len(today_open_market))

[Getting data:]############################################################
len(today_open_market) 3608
Wall time: 55.6 s


## 执行卖出


## 计算买入信号

In [34]:
today = pd.to_datetime(time.strftime('%Y-%m-%d'))
today = today.to_period(freq='D')

benchmark_lastday_market = stock_df_dict[BENCHMARK].loc[:today].iloc[-1]
benchmark_today_market = stock_df_dict[BENCHMARK].loc[:today].iloc[-1]

target_today_market = today_open_market.copy()
target_today_market.set_index(keys='code', inplace=True)
# target_today_market[:10]
# stock_df_dict['600000']
# target_today_market.loc['600000']
buy_list = []

# 趋势交易，只在好行情时买入
is_market_up = True
if IS_MARKETUP:
    if benchmark_lastday_market.MA60 < benchmark_lastday_market.MA180:
        print('BENCHMARK', BENCHMARK)
        print('LASTDAY', benchmark_lastday_market.name)
        print('OPEN', benchmark_lastday_market.open)
        print('MA60', benchmark_today_market.MA60)
        print('MA180', benchmark_today_market.MA180)
        print('MARKET IS DOWN, STOP BUYING!')
        is_market_up = False


# 遍历标的，判断和执行买入
for symbol in TARGET:
#     print(symbol)
    
    if not is_market_up:
        pass
#         break

    if len(stock_df_dict[symbol]) == 0:
        continue

    today_market = target_today_market.loc[symbol]
#     today_market
    lastday_market = stock_df_dict[symbol].loc[:today].iloc[-1]
#     lastday_market
    
#     if today - lastday_market.name > 3:
#         continue

    # 突破上行趋势，就买一份
    is_buy = False

    # 指数就不要过滤器了
    if True:
        if today_market.open >= lastday_market['ROLLING_%d_MAX' % TURTLE_LONG_BUY_N]:
            is_buy = True
            buy_reason = 'LONG'
        elif False and today_market.open >= lastday_market['ROLLING_%d_MAX' % TURTLE_SHORT_BUY_N]:
            is_buy = True
            buy_reason = 'SHORT'

    # 加入购买信号列表
    if is_buy:
        buy_list.append(symbol)


if IS_RANDOM_BUY:
    random.shuffle(buy_list)
else:
    tmp_list = []
    for symbol in buy_list:
        try:
            return_lastyear = stock_df_dict[symbol][:today].iloc[-1].open / stock_df_dict[symbol][:today].iloc[-250].open - 1
        except:
            return_lastyear = stock_df_dict[symbol][:today].iloc[-1].open / stock_df_dict[symbol][:today].iloc[1].open - 1
        tmp_list.append((return_lastyear, symbol))
    tmp_list = sorted(tmp_list, reverse=True)
#     buy_list = [x[1] for x in tmp_list if x[0]>0]
    buy_list = [x[1] for x in tmp_list]
    buy_list = tmp_list


buy_list_df = None
buy_list_df = pd.DataFrame(columns=[
    'today', 'symbol', 'name', 'now_open', 'RO60_MAX', 'p_date', 'p_open', 'p_close',
    'MA30', 'MA60', 'MA180', 'now_vol', 'ret_lastyear', 'per', 'pb'
])
for return_lastyear, symbol in buy_list:
    today_market = target_today_market.loc[symbol]
    lastday_market = stock_df_dict[symbol].loc[:today].iloc[-1]
    buy_list_df = buy_list_df.append(
        {
            'today': today,
            'symbol': symbol,
            'name': today_market['name'],
            'now_open': today_market.open,
            'RO60_MAX': lastday_market.ROLLING_60_MAX,
            'p_date': lastday_market.name,
            'p_open': lastday_market.open,
            'p_close': lastday_market.close,
            'MA30': lastday_market.MA30,
            'MA60': lastday_market.MA60,
            'MA180': lastday_market.MA180,
            'now_vol': int(today_market.volume / 10000),
            'ret_lastyear': return_lastyear,
            'per': today_market.per,
            'pb': today_market.pb,
        },
        ignore_index = True
    )


'''今天买入目标'''
buy_list_df

for i in range(len(buy_list_df)):
    symbol = buy_list_df.iloc[i].symbol
    name = buy_list_df.iloc[i]['name']
    df = stock_df_dict[symbol].copy()
    df.reset_index(drop=False, inplace=True)
    df['date'] = df['date'].apply(lambda x: x.to_timestamp().to_datetime64())
    df.set_index('date', inplace=True)
    display_charts(
        df, chart_type='stock', title='%s %s' % (symbol, name), 
        y=['open', 'ROLLING_60_MAX', 'ROLLING_60_MIN', 'MA60', 'MA180'], 
        figsize=(900, 600)
    )

BENCHMARK 399300
LASTDAY 2018-08-24
OPEN 3308.48
MA60 3502.7274999999995
MA180 3837.8354999999983
MARKET IS DOWN, STOP BUYING!


'今天买入目标'

,today,symbol,name,now_open,RO60_MAX,p_date,p_open,p_close,MA30,MA60,MA180,now_vol,ret_lastyear,per,pb
0,2018-08-27,601006,大秦铁路,9.05,9.040,2018-08-24,8.97,9.01,8.837000,8.599317,8.511633,3297,0.061915,10.133,1.310
1,2018-08-27,600583,海油工程,6.06,5.908,2018-08-24,5.89,6.02,5.350333,5.342933,5.955089,3077,-0.038524,56.727,1.218
2,2018-08-27,601229,上海银行,11.53,11.450,2018-08-24,11.23,11.44,11.082000,10.933950,10.634367,2744,-0.094282,5.878,0.948
3,2018-08-27,600339,中油工程,4.82,4.780,2018-08-24,4.74,4.84,4.382800,4.326667,4.621550,1676,-0.317593,41.461,1.242


## 人工执行/记录买入

# 每天收盘后

## 下载数据到本地

In [10]:
%%time

IS_DOWNLOAD_DATA = False
ignore = True

for symbol in TARGET + [BENCHMARK]:
    if not IS_DOWNLOAD_DATA:
        break

    # 沪深300指数
    if symbol == '399300':
        df = ts.get_k_data(code=symbol, index=True, start='2017-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s_lite.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # A股
    elif symbol in HS300:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2017-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s_lite.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')

    print(datafile)
    time.sleep(1)

Wall time: 0 ns
